<a href="https://colab.research.google.com/github/JahnaviKomatireddy/ImageClassification/blob/main/imgclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from skimage.transform import resize
from skimage.io import imread

from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import UndefinedMetricWarning

# Ignore undefined metric warnings globally
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Category input
print("Type 'y' to give custom categories or type 'n' to go with default: Cars, Ice Cream Cone, Cricket Ball")
while True:
    check = input().strip().lower()
    if check == 'n':
        Categories = ['Cars', 'Ice cream cone', 'Cricket ball']
        break
    elif check == 'y':
        Categories = []
        n = int(input("Enter how many types of images you want to classify: "))
        print(f"Please enter {n} category names:")
        for _ in range(n):
            name = input().strip()
            Categories.append(name)
        print(f"Make sure you upload all the {n} category image folders to Google Drive under '/content/drive/MyDrive/' with the exact names.")
        break
    else:
        print("Please give a valid input (y/n)")

# Load images and preprocess
flat_data_arr = []
target_arr = []

for i in Categories:
    print(f'Loading category: {i}')
    path = os.path.join(datadir, i)  # <- full path to each category
    if not os.path.exists(path):
        print(f"Folder not found: {path}")
        continue
    for img in os.listdir(path):
        try:
            img_array = imread(os.path.join(path, img))
            img_resized = resize(img_array, (150, 150, 3))
            flat_data_arr.append(img_resized.flatten())
            target_arr.append(Categories.index(i))
        except:
            print(f"Failed to load image: {img}")
    print(f'Loaded category: {i} successfully.')


flat_data = np.array(flat_data_arr)
target = np.array(target_arr)
df = pd.DataFrame(flat_data)
df['Target'] = target

# Train-test split
x = df.iloc[:, :-1]
y = df.iloc[:, -1]
unique, counts = np.unique(y, return_counts=True)
print("Class distribution:", dict(zip(unique, counts)))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=77)
print('Data split successfully.')

# Model training using GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.0001, 0.001, 0.1, 1], 'kernel': ['rbf', 'poly']}
svc = svm.SVC(probability=True)
print("Training the model... This may take a few minutes.")
cv = StratifiedKFold(n_splits=2)
model = GridSearchCV(svc, param_grid,cv=cv)
model.fit(x_train, y_train)
print("Model training complete.")
print("Best parameters found:", model.best_params_)

# Evaluation
y_pred = model.predict(x_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# Save model
pickle.dump(model, open('img_model.p', 'wb'))
print("Model saved as 'img_model.p'.")

import warnings
from sklearn.metrics import classification_report





# Suppress specific sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# Continue with your existing code for model training
model = GridSearchCV(svc, param_grid, cv=cv)
model.fit(x_train, y_train)

# Get classification report with zero_division set to avoid warnings
y_pred = model.predict(x_test)
report = classification_report(y_test, y_pred, zero_division=0)
print(report)

# Prediction on new image
model = pickle.load(open('img_model.p', 'rb'))
url = input('Enter URL or path of the image: ')
img = imread(url)
plt.imshow(img)
plt.axis('off')
plt.show()

img_resize = resize(img, (150, 150, 3))
l = [img_resize.flatten()]
probability = model.predict_proba(l)

for ind, val in enumerate(Categories):
    print(f'{val}: {probability[0][ind]*100:.2f}%')

prediction = model.predict(l)[0]
print(f"Predicted image is: {Categories[prediction]}")
# After predicting
print(classification_report(y_test, y_pred, zero_division=0))
print(f"Is the image a {Categories[prediction]}? (y/n)")

while True:
    feedback = input().strip().lower()
    if feedback in ['y', 'n']:
        break
    print("Please enter either 'y' or 'n'.")


# Online learning if wrong prediction
if feedback == 'n':
    print("What is the correct category?")
    for i, cat in enumerate(Categories):
        print(f"{i}: {cat}")
    while True:
        try:
            k = int(input("Enter the correct category index: "))
            if 0 <= k < len(Categories):
                break
            else:
                print("Invalid index. Try again.")
        except ValueError:
            print("Enter a valid number.")

    print("Updating the model with the new image...")
    flat_data_arr.append(l[0])
    target_arr.append(k)
    flat_data = np.array(flat_data_arr)
    target = np.array(target_arr)
    df1 = pd.DataFrame(flat_data)
    df1['Target'] = target
    x1 = df1.iloc[:, :-1]
    y1 = df1.iloc[:, -1]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.50, random_state=77)


    # Train new model with same params
    model1 = GridSearchCV(svc, {k: [v] for k, v in model.best_params_.items()})
    model1.fit(x_train1, y_train1)
    y_pred1 = model1.predict(x_test1)
    print(f"Updated model accuracy: {accuracy_score(y_test1, y_pred1) * 100:.2f}%")
    pickle.dump(model1, open('img_model.p', 'wb'))
    print("Updated model saved.")
    print("Thank you for your feedback!")

Type 'y' to give custom categories or type 'n' to go with default: Cars, Ice Cream Cone, Cricket Ball
